# Aqua Update Demos: Operator Fun

_donny@, 13-Mar-20_

In [1]:
import numpy as np
from qiskit.aqua.operators import I, X, Y, Z, H, CX, Zero, ListOp, PauliExpectation, PauliTrotterEvolution
from qiskit.circuit import Parameter

## Construct an H2 Hamiltonian

In [2]:
two_qubit_H2 =  (-1.0523732 * I^I) + \
                (0.39793742 * I^Z) + \
                (-0.3979374 * Z^I) + \
                (-0.0112801 * Z^Z) + \
                (0.18093119 * X^X)

## Evolve a Bell state by Our Hamiltonian

OpFlow fully supports parameterization, so we can use a parameter for our evolution time here. Notice that there's no "evolution time" argument in any function. OpFlow exponentiates whatever operator we tell it to, and if we choose to multiply the operator by an evolution time, $e^{iHt}$, that will be refected in our exponentiation parameters. This is not some trick to make it look like Physics - it actually works this way under the hood.

In [3]:
# Meaningless state
bell = CX @ (H^I) @ Zero
# We can also do CX @ (Plus ^ Zero)
evo_time = Parameter('θ')
wf = (evo_time*two_qubit_H2).exp_i() @ bell
trot = PauliTrotterEvolution(trotter_mode='trotter').convert(wf)
trot.to_circuit().draw(fold=1000)

┌───┐┌───┐┌──────────────────┐┌───┐┌───┐     ┌───┐┌──────────────────┐┌───┐┌──────────────────┐
q_0: ───────■──┤ H ├┤ X ├┤ RZ(0.18093119*θ) ├┤ X ├┤ H ├─────┤ X ├┤ RZ(-0.0112801*θ) ├┤ X ├┤ RZ(0.39793742*θ) ├
     ┌───┐┌─┴─┐├───┤└─┬─┘└──────────────────┘└─┬─┘├───┤┌───┐└─┬─┘└──────────────────┘└─┬─┘├──────────────────┤
q_1: ┤ H ├┤ X ├┤ H ├──■────────────────────────■──┤ H ├┤ I ├──■────────────────────────■──┤ RZ(-0.3979374*θ) ├
     └───┘└───┘└───┘                              └───┘└───┘                              └──────────────────┘

We can bind our parameter to the operator if we so choose, and it will recursively bind into the circuit.

In [4]:
bound = trot.bind_parameters({evo_time: .5})
bound.to_circuit().draw(fold=1000)

┌───┐┌───┐┌──────────────┐┌───┐┌───┐     ┌───┐┌──────────────┐┌───┐┌─────────────┐ 
q_0: ───────■──┤ H ├┤ X ├┤ RZ(0.090466) ├┤ X ├┤ H ├─────┤ X ├┤ RZ(-0.00564) ├┤ X ├┤ RZ(0.19897) ├─
     ┌───┐┌─┴─┐├───┤└─┬─┘└──────────────┘└─┬─┘├───┤┌───┐└─┬─┘└──────────────┘└─┬─┘├─────────────┴┐
q_1: ┤ H ├┤ X ├┤ H ├──■────────────────────■──┤ H ├┤ I ├──■────────────────────■──┤ RZ(-0.19897) ├
     └───┘└───┘└───┘                          └───┘└───┘                          └──────────────┘

## Now that we have a state, let's measure the energy of the state

In [5]:
from qiskit import BasicAer
backend = BasicAer.get_backend('qasm_simulator')

PauliExpectation(operator=two_qubit_H2, backend=backend).compute_expectation(bound)

(-0.6537181160546873+0j)

### Why We Call it OpFlow

The power of OpFlow is that everything is an Operator, so we can ask PauliExpectation for the intermediate operator it generated before it used a Circuit sampler to replace the circuits with dicts:

In [6]:
expect_op = PauliExpectation(operator=two_qubit_H2, backend=backend).expectation_op(bound)
print(expect_op)

SummedOp(
[ComposedOp(
[Measurement(SummedOp(
[0.18093119 * ZZ,
-1.0523732 * II])),
StateFunction(
               ┌───┐┌───┐┌────────────────────────┐┌───┐┌───┐┌───┐»
q_0: ───────■──┤ H ├┤ X ├┤ Rz(0.0904655950000000) ├┤ X ├┤ H ├┤ X ├»
     ┌───┐┌─┴─┐├───┤└─┬─┘└────────────────────────┘└─┬─┘├───┤└─┬─┘»
q_1: ┤ H ├┤ X ├┤ H ├──■──────────────────────────────■──┤ H ├──■──»
     └───┘└───┘└───┘                                    └───┘     »
«     ┌──────────────────────────┐┌───┐┌───────────────────────┐ ┌───┐
«q_0: ┤ Rz(-0.00564005000000000) ├┤ X ├┤ Rz(0.198968710000000) ├─┤ H ├
«     └──────────────────────────┘└─┬─┘├───────────────────────┴┐├───┤
«q_1: ──────────────────────────────■──┤ Rz(-0.198968700000000) ├┤ H ├
«                                      └────────────────────────┘└───┘
)]),
ComposedOp(
[Measurement(SummedOp(
[0.39793742 * IZ,
-0.3979374 * ZI,
-0.0112801 * ZZ])),
StateFunction(
               ┌───┐┌───┐┌────────────────────────┐┌───┐┌───┐┌───┐»
q_0: ───────■──┤ H ├┤ X ├┤ R

### We can just as easily take the Expectation over a _vector_ of Pauli Operators.

In [7]:
ham_list = ListOp([X^X, Y^Y, Z^Z, two_qubit_H2])
expect = PauliExpectation(operator=ham_list, backend=backend).compute_expectation(bound)
expect

[(0.04492187500000017+0j),
 (-0.0078124999999999445+0j),
 (0.021484375+0j),
 (-0.6508426385937498+0j)]

### We Can Even Take the Expectation of a vector of Observables over a vector of StateFns

In [8]:
# Here we're using PauliExpectation's param argument instead of passing the bound ListOp below in case we can 
# take advantage of late binding / parameterized Qobj
params = {evo_time: [.5, 1.0, 1.5]}
expects = PauliExpectation(operator=ham_list, backend=backend).compute_expectation(trot, params=params)
np.real(np.around(expects, decimals=3))

array([[ 0.047, -0.027, -0.008, -0.673],
       [ 0.086,  0.033, -0.02 , -0.643],
       [ 0.096,  0.109, -0.014, -0.636]])

### Parameter binding also supports binding lists

In [9]:
print(trot.bind_parameters(params))

ListOp(
[StateFunction(
               ┌───┐┌───┐┌────────────────────────┐┌───┐┌───┐┌───┐»
q_0: ───────■──┤ H ├┤ X ├┤ Rz(0.0904655950000000) ├┤ X ├┤ H ├┤ X ├»
     ┌───┐┌─┴─┐├───┤└─┬─┘└────────────────────────┘└─┬─┘├───┤└─┬─┘»
q_1: ┤ H ├┤ X ├┤ H ├──■──────────────────────────────■──┤ H ├──■──»
     └───┘└───┘└───┘                                    └───┘     »
«     ┌──────────────────────────┐┌───┐┌───────────────────────┐ 
«q_0: ┤ Rz(-0.00564005000000000) ├┤ X ├┤ Rz(0.198968710000000) ├─
«     └──────────────────────────┘└─┬─┘├───────────────────────┴┐
«q_1: ──────────────────────────────■──┤ Rz(-0.198968700000000) ├
«                                      └────────────────────────┘
),
StateFunction(
               ┌───┐┌───┐┌───────────────────────┐┌───┐┌───┐┌───┐»
q_0: ───────■──┤ H ├┤ X ├┤ Rz(0.180931190000000) ├┤ X ├┤ H ├┤ X ├»
     ┌───┐┌─┴─┐├───┤└─┬─┘└───────────────────────┘└─┬─┘├───┤└─┬─┘»
q_1: ┤ H ├┤ X ├┤ H ├──■─────────────────────────────■──┤ H ├──■──»
     └───┘└───┘└───┘